In [ ]:
from utils import petab as pet
import numpy as np
%load_ext autoreload
%autoreload 2

### Define Petab Observables

In [ ]:
observables_df = pet.define_observables({
    'xA': 'xA',
    'xB': 'xB',
    'valid_xA': 'is_valid_xA*xA',
    
}, noise_value=0.02)

observables_df

### Define Petab Conditions (Initial conditions)

In [ ]:
fA = np.array([0.25, 0.5, 0.75], dtype=float)
cM = np.array([1.0, 1.0, 1.0], dtype=float)
R0 = np.array([0.01, 0.01, 0.01], dtype=float)
A0 = fA * cM
B0 = (1 - fA) * cM

# Define conditions dataframe
conditions_df = pet.define_conditions(
    init_conditions={
        'A0': list(A0),
        'B0': list(B0),
        # 'R': list(R0),
        # 'A': list(A0),
        # 'B': list(B0),
    }
)
conditions_df

### Define Petab Parameters

In [ ]:
from petab.v1.C import LOG10, LIN
from utils.petab import PetabParameter as Param
params = [
    Param(id='rA', scale=LOG10, bounds=(1e-3, 1e3), nominal_value=1.0, estimate=True),
    Param(id='rB', scale=LOG10, bounds=(1e-3, 1e3), nominal_value=1.0, estimate=True),
    Param(id='rX', scale=LOG10, bounds=(1e-3, 1e3), nominal_value=1.0, estimate=False),
    Param(id='KAA', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='KAB', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='KBA', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='KBB', scale=LIN, bounds=(0, 1),  nominal_value=0.0, estimate=False),
    Param(id='kpAA', scale=LOG10, bounds=(1e0, 1e6), nominal_value=1.0, estimate=False),
]
parameters_df = pet.define_parameters(params)
parameters_df

### Define Reversible Copolymerization Model

In [ ]:
import utils.sbml as sbml
import utils.amici as am
from models.sbml_models import CRP2_v1, CRP2_CPE


sbml_model_filepath = sbml.write_model(CRP2_CPE)

validator = sbml.validateSBML(ucheck=False)
validator.validate(sbml_model_filepath)
model_name, model_output_dir = am.compile_amici_model(sbml_model_filepath, observables_df, verbose=False)
model = am.load_amici_model(model_name, model_output_dir)


### Define Petab Measurements

In [ ]:
# Define synthetic measurements
true_params = {
    # 'kpAA': 1.0,
    'rA': 1.1,
    'rB': 5.,
    'rX': 1.0,
    'KAA': 0.0,
    'KAB': 0.0,
    'KBA': 0.0,
    'KBB': 0.0,
}
model = am.set_model_parameters(model, true_params)

# timepoints = list(np.linspace(0, 500, 20))
timepoints = list(np.arange(0, 1, 0.1, dtype=float))

measurements_df, rdatas = pet.define_measurements(
    amici_model=model, 
    timepoints=timepoints,
    conditions_df=conditions_df, 
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    debug_return_rdatas=True
)

measurements_df

### Write Petab problem

In [ ]:
yaml_filepath = pet.write_petab_files(
    sbml_model_filepath,
    parameters_df,
    observables_df,
    conditions_df,
    measurements_df,
)
yaml_filepath

### Compare with Copolymer Equation

In [ ]:
from models.cpe_models import CPE

